In [1]:
import pandas as pd
import os

from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
import numpy as np

from math import log
from collections import Counter

import json


In [2]:
print(sklearn.__version__)

1.0.2


In [3]:
# Запускать только в первый раз, чтобы скачать
download('punkt')
download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ivan7\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivan7\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def lemmatize(a_text):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = [morph.parse(item)[0].normal_form for item in a_tokens]
    a_lemmatized = ' '.join([token for token in a_lemmatized if token.isalpha()])
    return a_lemmatized


def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]


def add_best_words(row):
    i = row['Unnamed: 0']
    article_vector = articles_tfidf[i, :]
    words = get_top_tf_idf_words(article_vector, feature_names, 10)
    return words


def enumerate_words(lists_of_keywords):
    i = 0
    words_indices = {}
    for keywords in lists_of_keywords:
        for word in keywords:
            if word not in words_indices:
                words_indices[word] = i
                i += 1
                
    return words_indices


def create_idf(lists_of_keywords):
    count_dictionary = Counter()
    for keywords in lists_of_keywords:
        words = set(keywords)
        for word in words:
            count_dictionary[word] += 1

    idf = {}
    for word, count in count_dictionary.items():
        idf[word] = log(len(lists_of_keywords) / count)
        # your code here
    return idf


def create_tf_idf(idf, lists_of_keywords, words_indices):
    tf_idf = [[0] * len(idf) for i in range(len(lists_of_keywords))]
    for n, sentence in enumerate(lists_of_keywords):
        words_counter = Counter(sentence)
        sentence_length = len(sentence)
        for word in words_counter:
            word_index = words_indices[word]
            word_tf = words_counter[word] / sentence_length
            word_tf_idf = word_tf * idf[word]

            tf_idf[n][word_index] = word_tf_idf

    return tf_idf

In [9]:
# Читаем необработанный корпус, где 3 колонки
df = pd.read_csv('Корпус Shape.csv')

In [ ]:
# Создаём стоп слова и анализатор
stops = stopwords.words('russian')
morph = MorphAnalyzer()

In [10]:
# Создаём колонку с обработанным текстом
df['lemmatized_text'] = df.text.apply(lemmatize)

In [11]:
# Создаём векторизатор
tfidf = TfidfVectorizer(
    analyzer="word", # анализировать по словам или по символам (char)
        stop_words=stops, # передаём список стоп-слов для русского из NLTK
    ngram_range=(1, 1)
)

In [12]:
articles_tfidf = tfidf.fit_transform(df['lemmatized_text'])

In [13]:
feature_names = np.array(tfidf.get_feature_names())
feature_words = []
for i, article in enumerate(df.lemmatized_text):
    article_vector = articles_tfidf[i, :]
    words = get_top_tf_idf_words(article_vector, feature_names, 10)



C:\Users\ivan7\PycharmProjects\AnastasiaPythonBot\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
# Создаём колонку с ключевыми словами
df['keywords'] = df.apply(add_best_words, axis=1)

In [15]:
words_indices = enumerate_words(df.keywords)

In [16]:
words_indices = enumerate_words(df.keywords)
idf = create_idf(df['keywords'])
tf_idf = create_tf_idf(idf, df['keywords'], words_indices)

# Создаём колонку с tf_idf векторами
df['tf_idf'] = tf_idf

In [17]:
# Сохранение словаря в файл
with open('words_indices.json', 'w') as f:
    json.dump(words_indices, f)
    


In [39]:
# Сохранение словаря в файл
with open('idf_data.json', 'w') as f:
    json.dump(idf, f)

In [18]:
# Чтение соваря из файла
with open('words_indices.json', 'r') as f:
    words_indices_loaded = json.load(f)


In [37]:
df.to_csv('processed_data.csv', index_label='index')


In [31]:
df_2 = pd.read_csv('processed_data2.csv', index_col='index')

In [35]:
df.drop(columns=['Unnamed: 0'])

,link,title,text,lemmatized_text,keywords,tf_idf
0,https://shape.ru/radosti-zhizni/transformatsiy...,"Как стать экспертом, которого все хотят","Даже если вы — вполне успешный профи, с прекра...",даже если вы вполне успешный профи с прекрасны...,"[pr, клиент, репутация, конкурент, катерина, к...","[0.5440250862436703, 0.613339804299665, 0.6133..."
1,https://shape.ru/radosti-zhizni/transformatsiy...,Если вы трудоголик: как отучить себя работать ...,Многие люди выходят на работу в свой законный ...,многие человек выходить на работа в свой закон...,"[выходной, рабочий, законный, отдых, граница, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5034785754328..."
2,https://shape.ru/radosti-zhizni/motivatsiya/50...,Рай для двоих: 7 направлений для путешествия-с...,Романтические путешествия сближают и помогают ...,романтический путешествие сближать и помогать ...,"[пляж, бронировать, остров, сочи, отель, путеш...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,https://shape.ru/radosti-zhizni/transformatsiy...,"5 советов, которые помогут научиться достигать...","Сколько раз мы сталкивались с тем, что хотели ...",сколько раз мы сталкиваться с тем что хотеть ч...,"[цель, самозванец, идти, желаемый, кузав, стан...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,https://shape.ru/radosti-zhizni/transformatsiy...,Что в жизни важнее — семья или карьера?,Нужно ли женщине выбирать между семьей и карье...,нужно ли женщина выбирать между семья и карьер...,"[семья, женщина, карьера, совмещать, ребёнок, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
456,https://shape.ru/radosti-zhizni/motivatsiya/13...,Где провести выходные,Летний уикенд: 20-21июня\r\nБесплатная йога\r\...,летний уикенд бесплатный йог июнь в весь мир о...,"[фестиваль, вднх, пляжный, июнь, бесплатный, г...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
457,https://shape.ru/radosti-zhizni/motivatsiya/24...,"КАК ГОВОРИТЬ С БОССОМ, ЧТОБЫ ОН ВАС УСЛЫШАЛ",Боитесь просить повышения по службе или стесня...,бояться просить повышение по служба или стесня...,"[босс, предложение, информация, разговор, прос...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
458,https://shape.ru/radosti-zhizni/motivatsiya/13...,Как хорошо получаться на фото,Главные правила для фотомоделиПравило №1. Прин...,главный правило для фотомоделиправить принимат...,"[фотограф, глаз, подбородок, обзавестись, прав...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
459,https://shape.ru/radosti-zhizni/transformatsiy...,Как знаменитости худеют после родов?,1. Как худела после родов Бейонсе\r\nСпустя 4 ...,как худеть после роды бейонс спустя месяц посл...,"[роды, алессандра, худеть, пинк, бейонс, миран...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [38]:
df


,Unnamed: 0,link,title,text,lemmatized_text,keywords,tf_idf
0,0,https://shape.ru/radosti-zhizni/transformatsiy...,"Как стать экспертом, которого все хотят","Даже если вы — вполне успешный профи, с прекра...",даже если вы вполне успешный профи с прекрасны...,"[pr, клиент, репутация, конкурент, катерина, к...","[0.5440250862436703, 0.613339804299665, 0.6133..."
1,1,https://shape.ru/radosti-zhizni/transformatsiy...,Если вы трудоголик: как отучить себя работать ...,Многие люди выходят на работу в свой законный ...,многие человек выходить на работа в свой закон...,"[выходной, рабочий, законный, отдых, граница, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5034785754328..."
2,2,https://shape.ru/radosti-zhizni/motivatsiya/50...,Рай для двоих: 7 направлений для путешествия-с...,Романтические путешествия сближают и помогают ...,романтический путешествие сближать и помогать ...,"[пляж, бронировать, остров, сочи, отель, путеш...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,https://shape.ru/radosti-zhizni/transformatsiy...,"5 советов, которые помогут научиться достигать...","Сколько раз мы сталкивались с тем, что хотели ...",сколько раз мы сталкиваться с тем что хотеть ч...,"[цель, самозванец, идти, желаемый, кузав, стан...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,https://shape.ru/radosti-zhizni/transformatsiy...,Что в жизни важнее — семья или карьера?,Нужно ли женщине выбирать между семьей и карье...,нужно ли женщина выбирать между семья и карьер...,"[семья, женщина, карьера, совмещать, ребёнок, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
456,456,https://shape.ru/radosti-zhizni/motivatsiya/13...,Где провести выходные,Летний уикенд: 20-21июня\r\nБесплатная йога\r\...,летний уикенд бесплатный йог июнь в весь мир о...,"[фестиваль, вднх, пляжный, июнь, бесплатный, г...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
457,457,https://shape.ru/radosti-zhizni/motivatsiya/24...,"КАК ГОВОРИТЬ С БОССОМ, ЧТОБЫ ОН ВАС УСЛЫШАЛ",Боитесь просить повышения по службе или стесня...,бояться просить повышение по служба или стесня...,"[босс, предложение, информация, разговор, прос...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
458,458,https://shape.ru/radosti-zhizni/motivatsiya/13...,Как хорошо получаться на фото,Главные правила для фотомоделиПравило №1. Прин...,главный правило для фотомоделиправить принимат...,"[фотограф, глаз, подбородок, обзавестись, прав...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
459,459,https://shape.ru/radosti-zhizni/transformatsiy...,Как знаменитости худеют после родов?,1. Как худела после родов Бейонсе\r\nСпустя 4 ...,как худеть после роды бейонс спустя месяц посл...,"[роды, алессандра, худеть, пинк, бейонс, миран...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df_2.to_csv('processed_data.csv')
